# Phase 0: Enviroment Setup

In [2]:
# Install required packages
!pip install pgmpy pandas numpy matplotlib seaborn networkx fuzzywuzzy python-Levenshtein kaggle nba_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.0/287.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 68.6 MB/s eta 0:00:00


In [3]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
from pgmpy.inference import VariableElimination
import warnings
warnings.filterwarnings('ignore')

print("✅ All packages installed and imported successfully!")

✅ All packages installed and imported successfully!


# Phase 1: Data Acquistion & Problem Formalization

## Phase 1.1: Install NBA API and Get Data


In [4]:
print("🚀 GETTING REAL NBA LINEUP DATA FROM OFFICIAL NBA API...")

# Install nba_api
!pip install nba_api

from nba_api.stats.endpoints import teamdashlineups
from nba_api.stats.static import teams
import pandas as pd

# Get all NBA teams
nba_teams = teams.get_teams()

# Create team dictionary
team_dict = {}
for team in nba_teams:
    team_name = team['full_name']
    team_id = team['id']
    team_dict[team_name] = team_id

print(f"✅ Found {len(team_dict)} NBA teams")

# Function to get lineups for a team
def get_lineups(team_id_i):
    try:
        lineup = teamdashlineups.TeamDashLineups(
            team_id=team_id_i,
            season='2023-24',  # Using 2023-24 for more complete data
            season_type_all_star='Regular Season',
            group_quantity=5,  # 5-man lineups
            per_mode_detailed='Totals'
        )
        df = lineup.get_data_frames()
        all_lineups = df[1]  # This contains the lineup data
        return all_lineups
    except Exception as e:
        print(f"❌ Error getting lineups for team {team_id_i}: {e}")
        return None

# Get lineups for all teams
print("\n📥 DOWNLOADING LINEUP DATA FOR ALL TEAMS...")
dataframes = []

for i, team_name in enumerate(team_dict.keys()):
    team_id_i = team_dict[team_name]
    print(f"   {i+1}/{len(team_dict)}: Getting {team_name}...")

    team_lineup = get_lineups(team_id_i)
    if team_lineup is not None and not team_lineup.empty:
        team_lineup['team'] = team_name
        team_lineup['team_id'] = team_id_i
        dataframes.append(team_lineup)

    # Add small delay to avoid overwhelming API
    import time
    time.sleep(0.5)

# Combine all team lineups
if dataframes:
    league_lineup = pd.concat(dataframes, ignore_index=True)

    # Process the lineup data
    league_lineup['players_list'] = league_lineup['GROUP_NAME'].str.split(' - ')

    print(f"\n✅ SUCCESS: Downloaded {len(league_lineup)} lineup combinations!")
    print(f"📊 Dataset shape: {league_lineup.shape}")

    # Save the data
    league_lineup.to_csv('nba_lineups_2024_api.csv', index=False)
    print("💾 Saved as 'nba_lineups_2024_api.csv'")

    # Show sample
    print("\n🔍 SAMPLE OF REAL NBA LINEUP DATA:")
    display(league_lineup[['GROUP_NAME', 'team', 'MIN', 'PLUS_MINUS', 'FG_PCT', 'FG3_PCT']].head(3))

else:
    print("❌ No lineup data could be downloaded")

🚀 GETTING REAL NBA LINEUP DATA FROM OFFICIAL NBA API...
✅ Found 30 NBA teams

📥 DOWNLOADING LINEUP DATA FOR ALL TEAMS...
   1/30: Getting Atlanta Hawks...
   2/30: Getting Boston Celtics...
   3/30: Getting Cleveland Cavaliers...
   4/30: Getting New Orleans Pelicans...
   5/30: Getting Chicago Bulls...
   6/30: Getting Dallas Mavericks...
   7/30: Getting Denver Nuggets...
   8/30: Getting Golden State Warriors...
   9/30: Getting Houston Rockets...
   10/30: Getting Los Angeles Clippers...
   11/30: Getting Los Angeles Lakers...
   12/30: Getting Miami Heat...
   13/30: Getting Milwaukee Bucks...
   14/30: Getting Minnesota Timberwolves...
   15/30: Getting Brooklyn Nets...
   16/30: Getting New York Knicks...
   17/30: Getting Orlando Magic...
   18/30: Getting Indiana Pacers...
   19/30: Getting Philadelphia 76ers...
   20/30: Getting Phoenix Suns...
   21/30: Getting Portland Trail Blazers...
   22/30: Getting Sacramento Kings...
   23/30: Getting San Antonio Spurs...
   24/30: Ge

,GROUP_NAME,team,MIN,PLUS_MINUS,FG_PCT,FG3_PCT
0,C. Capela - D. Murray - T. Young - S. Bey - J....,Atlanta Hawks,288.680000,-88.0,0.446,0.312
1,C. Capela - D. Murray - T. Young - D. Hunter -...,Atlanta Hawks,176.911667,8.0,0.468,0.384
2,C. Capela - D. Murray - T. Young - D. Hunter -...,Atlanta Hawks,171.505000,-26.0,0.464,0.367


## Phase 1.2: Analyze the API Data Structure

In [5]:
print("🔬 ANALYZING NBA API DATA STRUCTURE...")

try:
    lineup_data = pd.read_csv('nba_lineups_2024_api.csv')

    print("📋 COLUMNS AVAILABLE:")
    for col in lineup_data.columns:
        print(f"   - {col}")

    print("\n🎯 VARIABLES FOR OUR BAYESIAN NETWORK:")

    # Check for critical variables
    critical_vars = {
        'Efficiency (Target)': ['PLUS_MINUS', 'PTS'],
        'Shooting': ['FG_PCT', 'FG3_PCT', 'EFG_PCT'],
        'Playmaking': ['AST', 'AST_PCT'],
        'Rebounding': ['OREB', 'DREB', 'REB'],
        'Turnovers': ['TOV', 'TOV_PCT']
    }

    available_cols = lineup_data.columns.tolist()

    for category, possible_vars in critical_vars.items():
        found = [var for var in possible_vars if var in available_cols]
        if found:
            print(f"   ✅ {category}: {found}")
        else:
            print(f"   ❌ {category}: Not found")

    print(f"\n📊 Dataset info: {lineup_data.shape}")
    print(f"👥 Unique lineups: {lineup_data['GROUP_NAME'].nunique()}")

except Exception as e:
    print(f"❌ Error analyzing data: {e}")

🔬 ANALYZING NBA API DATA STRUCTURE...
📋 COLUMNS AVAILABLE:
   - GROUP_SET
   - GROUP_ID
   - GROUP_NAME
   - GP
   - W
   - L
   - W_PCT
   - MIN
   - FGM
   - FGA
   - FG_PCT
   - FG3M
   - FG3A
   - FG3_PCT
   - FTM
   - FTA
   - FT_PCT
   - OREB
   - DREB
   - REB
   - AST
   - TOV
   - STL
   - BLK
   - BLKA
   - PF
   - PFD
   - PTS
   - PLUS_MINUS
   - GP_RANK
   - W_RANK
   - L_RANK
   - W_PCT_RANK
   - MIN_RANK
   - FGM_RANK
   - FGA_RANK
   - FG_PCT_RANK
   - FG3M_RANK
   - FG3A_RANK
   - FG3_PCT_RANK
   - FTM_RANK
   - FTA_RANK
   - FT_PCT_RANK
   - OREB_RANK
   - DREB_RANK
   - REB_RANK
   - AST_RANK
   - TOV_RANK
   - STL_RANK
   - BLK_RANK
   - BLKA_RANK
   - PF_RANK
   - PFD_RANK
   - PTS_RANK
   - PLUS_MINUS_RANK
   - SUM_TIME_PLAYED
   - team
   - team_id
   - players_list

🎯 VARIABLES FOR OUR BAYESIAN NETWORK:
   ✅ Efficiency (Target): ['PLUS_MINUS', 'PTS']
   ✅ Shooting: ['FG_PCT', 'FG3_PCT']
   ✅ Playmaking: ['AST']
   ✅ Rebounding: ['OREB', 'DREB', 'REB']
   ✅ Turno

## Phase 1.3: Integration with Kaggle Data

In [6]:
# === PHASE 1.3 FIXED: USE ONLY NBA API DATA ===
print("=== PHASE 1.3: PROPER NBA API DATA INTEGRATION ===")

# Load the NBA API data we just downloaded
print("📥 Loading NBA API lineup data...")
lineup_data = pd.read_csv('nba_lineups_2024_api.csv')

print(f"📊 Original NBA API data: {lineup_data.shape}")

# Select only the variables we need for our Bayesian network
print("\n🎯 SELECTING VARIABLES FOR BAYESIAN NETWORK:")
selected_vars = {
    'Efficiency': 'PLUS_MINUS',  # Net rating as efficiency proxy
    'Shooting_FG': 'FG_PCT',     # Field goal percentage
    'Shooting_3PT': 'FG3_PCT',   # 3-point percentage
    'Playmaking': 'AST',         # Assists
    'Turnovers': 'TOV',          # Turnovers
    'Offensive_Rebounding': 'OREB'  # Offensive rebounds
}

# Create our feature dataset
print("🔧 Creating feature dataset from NBA API data...")
feature_data = lineup_data[list(selected_vars.values())].copy()
feature_data.columns = list(selected_vars.keys())

print(f"📊 Feature dataset shape: {feature_data.shape}")

# Remove any missing values
feature_data = feature_data.dropna()
print(f"📊 After removing missing values: {feature_data.shape}")

# Check data quality
print("\n🔍 DATA QUALITY CHECK:")
print("Basic statistics:")
print(feature_data.describe())

# Check for reasonable ranges (basketball logic)
print("\n🏀 BASKETBALL LOGIC VALIDATION:")
print("Ranges should make sense for NBA:")
for col in feature_data.columns:
    min_val = feature_data[col].min()
    max_val = feature_data[col].max()
    print(f"  {col}: {min_val:.2f} to {max_val:.2f}")

# Verify we have enough data for discretization
print(f"\n📈 DATA SUFFICIENCY:")
print(f"  Total samples: {len(feature_data)}")
print(f"  Minimum required: ~1,000 (for 3^5=243 combinations)")
print(f"  Status: {'✅ SUFFICIENT' if len(feature_data) >= 1000 else '❌ INSUFFICIENT'}")

if len(feature_data) >= 1000:
    # Save the integrated data for Phase 2
    feature_data.to_csv('nba_api_integrated_data.csv', index=False)
    print("💾 Saved integrated data as 'nba_api_integrated_data.csv'")

    print("\n✅ PHASE 1.3 COMPLETED SUCCESSFULLY!")
    print("🎯 Using ONLY NBA API data for consistency")
    print("🚀 Ready for Phase 2: Data Preprocessing")
else:
    print("\n❌ INSUFFICIENT DATA - Need to collect more NBA API data")
    print("   Consider multiple seasons or different API endpoints")

=== PHASE 1.3: PROPER NBA API DATA INTEGRATION ===
📥 Loading NBA API lineup data...
📊 Original NBA API data: (7500, 59)

🎯 SELECTING VARIABLES FOR BAYESIAN NETWORK:
🔧 Creating feature dataset from NBA API data...
📊 Feature dataset shape: (7500, 6)
📊 After removing missing values: (7500, 6)

🔍 DATA QUALITY CHECK:
Basic statistics:
        Efficiency  Shooting_FG  Shooting_3PT   Playmaking    Turnovers  \
count  7500.000000  7500.000000   7500.000000  7500.000000  7500.000000   
mean      0.555867     0.473603      0.354854     7.921333     3.910133   
std      11.382578     0.151966      0.248258    21.286298     9.458745   
min     -88.000000     0.000000      0.000000     0.000000     0.000000   
25%      -5.000000     0.387000      0.200000     2.000000     1.000000   
50%       0.000000     0.476000      0.333000     4.000000     2.000000   
75%       5.000000     0.563000      0.500000     7.000000     4.000000   
max     282.000000     1.000000      1.000000   615.000000   228.000

# Phase 2: Data Preprocessing & Discretization


## Phase 2.1: Data Cleaning & Filtering

In [7]:
# === PHASE 2.1 UPDATED: CLEAN NBA API DATA ===
print("=== PHASE 2.1: CLEANING NBA API DATA ===")

# Load the integrated NBA API data
print("📥 Loading integrated NBA API data...")
nba_api_data = pd.read_csv('nba_api_integrated_data.csv')

print(f"📊 Dataset shape: {nba_api_data.shape}")
print(f"🎯 Columns: {list(nba_api_data.columns)}")

# The data is already clean (no missing values), but let's verify
print("\n🔍 DATA CLEANLINESS CHECK:")
print(f"Missing values: {nba_api_data.isnull().sum().sum()}")  # Should be 0
print(f"Duplicate rows: {nba_api_data.duplicated().sum()}")    # Should be minimal

# Check for extreme outliers that might skew discretization
print("\n📊 OUTLIER DETECTION:")
for col in nba_api_data.columns:
    Q1 = nba_api_data[col].quantile(0.25)
    Q3 = nba_api_data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = nba_api_data[(nba_api_data[col] < lower_bound) | (nba_api_data[col] > upper_bound)]
    print(f"  {col}: {len(outliers)} outliers ({len(outliers)/len(nba_api_data):.1%})")

print("\n✅ PHASE 2.1 COMPLETED!")
print("🚀 Ready for Phase 2.2: Feature Selection & Engineering")

=== PHASE 2.1: CLEANING NBA API DATA ===
📥 Loading integrated NBA API data...
📊 Dataset shape: (7500, 6)
🎯 Columns: ['Efficiency', 'Shooting_FG', 'Shooting_3PT', 'Playmaking', 'Turnovers', 'Offensive_Rebounding']

🔍 DATA CLEANLINESS CHECK:
Missing values: 0
Duplicate rows: 162

📊 OUTLIER DETECTION:
  Efficiency: 324 outliers (4.3%)
  Shooting_FG: 234 outliers (3.1%)
  Shooting_3PT: 343 outliers (4.6%)
  Playmaking: 777 outliers (10.4%)
  Turnovers: 628 outliers (8.4%)
  Offensive_Rebounding: 729 outliers (9.7%)

✅ PHASE 2.1 COMPLETED!
🚀 Ready for Phase 2.2: Feature Selection & Engineering


## Phase 2.2: Data Preprocessing & Engineering

In [8]:
# === PHASE 2.2 FIXED: PROPER RATE STATISTICS ===
print("=== PHASE 2.2 FIXED: PROPER RATE STATISTICS ===")

# We need the original lineup data with MINUTES to convert to rates
print("📥 Loading full NBA lineup data with minutes...")
lineup_data = pd.read_csv('nba_lineups_2024_api.csv')

print("🔧 Converting totals to per-minute rates...")

# Calculate rates per 48 minutes (standard NBA rate)
def calculate_rates(data):
    rates_data = data.copy()

    # Efficiency stays as PLUS_MINUS (already a rate)
    rates_data['Efficiency'] = data['PLUS_MINUS']

    # Convert totals to per-48-minute rates
    minutes = data['MIN']

    # Shooting percentages stay the same (already rates)
    rates_data['Shooting_FG'] = data['FG_PCT']
    rates_data['Shooting_3PT'] = data['FG3_PCT']

    # Playmaking: Assists per 48 minutes
    rates_data['Playmaking'] = (data['AST'] / minutes) * 48

    # Turnovers: Turnovers per 48 minutes (INVERTED - lower is better)
    rates_data['Turnovers'] = (data['TOV'] / minutes) * 48

    # Offensive Rebounding: Offensive rebounds per 48 minutes
    rates_data['Offensive_Rebounding'] = (data['OREB'] / minutes) * 48

    return rates_data

# Create rate-based features
rates_data = calculate_rates(lineup_data)

# Select only our 6 key variables
rates_data = rates_data[['Efficiency', 'Shooting_FG', 'Shooting_3PT',
                        'Playmaking', 'Turnovers', 'Offensive_Rebounding']]

# Remove any infinite/NaN values from division
rates_data = rates_data.replace([np.inf, -np.inf], np.nan).dropna()

print(f"📊 Rate-based dataset shape: {rates_data.shape}")

# Check new correlations
print("\n📊 FIXED CORRELATIONS WITH EFFICIENCY:")
corr_matrix = rates_data.corr()
efficiency_correlations = corr_matrix['Efficiency'].sort_values(ascending=False)

for feature, corr in efficiency_correlations.items():
    if feature != 'Efficiency':
        print(f"   {feature}: {corr:.3f}")

# Verify basketball logic is now correct
positive_expected = ['Shooting_FG', 'Shooting_3PT', 'Playmaking', 'Offensive_Rebounding']
negative_expected = ['Turnovers']

actual_positive = [f for f in efficiency_correlations.index
                  if f != 'Efficiency' and efficiency_correlations[f] > 0]
actual_negative = [f for f in efficiency_correlations.index
                  if f != 'Efficiency' and efficiency_correlations[f] < 0]

print(f"\n✅ Expected Positive: {positive_expected}")
print(f"✅ Expected Negative: {negative_expected}")
print(f"📊 Actual Positive: {actual_positive}")
print(f"📊 Actual Negative: {actual_negative}")

# Basketball logic validation
if 'Turnovers' in actual_negative:
    print("🎯 BASKETBALL LOGIC: Turnovers now negatively correlate with efficiency ✓")
else:
    print("❌ BASKETBALL LOGIC STILL BROKEN - Need further investigation")

# Save the corrected data
rates_data.to_csv('nba_api_corrected_rates.csv', index=False)
print("\n💾 Saved corrected rate-based data as 'nba_api_corrected_rates.csv'")

print("\n✅ PHASE 2.2 FIXED COMPLETED!")
print("🚀 Ready for Phase 2.3 with proper basketball logic")

=== PHASE 2.2 FIXED: PROPER RATE STATISTICS ===
📥 Loading full NBA lineup data with minutes...
🔧 Converting totals to per-minute rates...
📊 Rate-based dataset shape: (7500, 6)

📊 FIXED CORRELATIONS WITH EFFICIENCY:
   Shooting_FG: 0.329
   Playmaking: 0.274
   Shooting_3PT: 0.223
   Offensive_Rebounding: 0.022
   Turnovers: -0.141

✅ Expected Positive: ['Shooting_FG', 'Shooting_3PT', 'Playmaking', 'Offensive_Rebounding']
✅ Expected Negative: ['Turnovers']
📊 Actual Positive: ['Shooting_FG', 'Playmaking', 'Shooting_3PT', 'Offensive_Rebounding']
📊 Actual Negative: ['Turnovers']
🎯 BASKETBALL LOGIC: Turnovers now negatively correlate with efficiency ✓

💾 Saved corrected rate-based data as 'nba_api_corrected_rates.csv'

✅ PHASE 2.2 FIXED COMPLETED!
🚀 Ready for Phase 2.3 with proper basketball logic


## Phase 2.3: Discretization

In [9]:
# === PHASE 2.3: SMART DISCRETIZATION ===
print("=== PHASE 2.3: SMART DISCRETIZATION ===")

# Load the corrected rate-based data
print("📥 Loading corrected rate-based data...")
rates_data = pd.read_csv('nba_api_corrected_rates.csv')

print(f"📊 Dataset shape: {rates_data.shape}")
print("🎯 Variables to discretize: Efficiency, Shooting_FG, Shooting_3PT, Playmaking, Turnovers, Offensive_Rebounding")

# Define basketball-informed discretization thresholds
print("\n🏀 SETTING BASKETBALL-INFORMED THRESHOLDS:")

discretization_rules = {
    'Efficiency': {
        'description': 'Plus/Minus per game',
        'Low': ('Below -5', 'Negative impact'),
        'Medium': ('-5 to +5', 'Neutral impact'),
        'High': ('Above +5', 'Positive impact')
    },
    'Shooting_FG': {
        'description': 'Field Goal Percentage',
        'Low': ('Below 45%', 'Poor shooting'),
        'Medium': ('45% to 50%', 'Average shooting'),
        'High': ('Above 50%', 'Elite shooting')
    },
    'Shooting_3PT': {
        'description': '3-Point Percentage',
        'Low': ('Below 35%', 'Poor 3PT'),
        'Medium': ('35% to 40%', 'Average 3PT'),
        'High': ('Above 40%', 'Elite 3PT')
    },
    'Playmaking': {
        'description': 'Assists per 48 minutes',
        'Low': ('Below 15', 'Low playmaking'),
        'Medium': ('15 to 25', 'Average playmaking'),
        'High': ('Above 25', 'High playmaking')
    },
    'Turnovers': {
        'description': 'Turnovers per 48 minutes',
        'Low': ('Below 10', 'Good ball control'),  # Lower turnovers = better
        'Medium': ('10 to 15', 'Average ball control'),
        'High': ('Above 15', 'Poor ball control')  # Higher turnovers = worse
    },
    'Offensive_Rebounding': {
        'description': 'Offensive Rebounds per 48 minutes',
        'Low': ('Below 8', 'Poor offensive rebounding'),
        'Medium': ('8 to 12', 'Average offensive rebounding'),
        'High': ('Above 12', 'Elite offensive rebounding')
    }
}

# Apply discretization
print("\n🔧 APPLYING DISCRETIZATION...")
final_discretized_data = rates_data.copy()

for column in final_discretized_data.columns:
    if column == 'Efficiency':
        bins = [-float('inf'), -5, 5, float('inf')]
        labels = ['Low', 'Medium', 'High']
    elif column == 'Shooting_FG':
        bins = [-float('inf'), 0.45, 0.50, float('inf')]
        labels = ['Low', 'Medium', 'High']
    elif column == 'Shooting_3PT':
        bins = [-float('inf'), 0.35, 0.40, float('inf')]
        labels = ['Low', 'Medium', 'High']
    elif column == 'Playmaking':
        bins = [-float('inf'), 15, 25, float('inf')]
        labels = ['Low', 'Medium', 'High']
    elif column == 'Turnovers':
        bins = [-float('inf'), 10, 15, float('inf')]
        labels = ['Low', 'Medium', 'High']  # Lower turnovers = "Low" category (good)
    elif column == 'Offensive_Rebounding':
        bins = [-float('inf'), 8, 12, float('inf')]
        labels = ['Low', 'Medium', 'High']

    final_discretized_data[column] = pd.cut(final_discretized_data[column], bins=bins, labels=labels)

print("✅ DISCRETIZATION COMPLETED!")

# Check the distribution of discretized variables
print("\n📊 DISCRETIZED DISTRIBUTIONS:")
for column in final_discretized_data.columns:
    dist = final_discretized_data[column].value_counts(normalize=True).sort_index()
    print(f"{column}:")
    for state in ['Low', 'Medium', 'High']:
        count = final_discretized_data[column].value_counts().get(state, 0)
        percentage = dist.get(state, 0) * 100
        print(f"  {state}: {count} samples ({percentage:.1f}%)")

# Verify we have enough samples in each category
print("\n🔍 SAMPLE SUFFICIENCY CHECK:")
min_samples = 100  # Minimum samples per category for reliable learning
for column in final_discretized_data.columns:
    for state in ['Low', 'Medium', 'High']:
        count = (final_discretized_data[column] == state).sum()
        if count < min_samples:
            print(f"⚠️  {column}-{state}: Only {count} samples")
        else:
            print(f"✅ {column}-{state}: {count} samples")

# Save the final discretized data
final_discretized_data.to_csv('final_discretized_nba_data.csv', index=False)
print("\n💾 Saved final discretized data as 'final_discretized_nba_data.csv'")

print("\n✅ PHASE 2.3 COMPLETED!")
print("🚀 Ready for Phase 2.4: Save Processed Data")

=== PHASE 2.3: SMART DISCRETIZATION ===
📥 Loading corrected rate-based data...
📊 Dataset shape: (7500, 6)
🎯 Variables to discretize: Efficiency, Shooting_FG, Shooting_3PT, Playmaking, Turnovers, Offensive_Rebounding

🏀 SETTING BASKETBALL-INFORMED THRESHOLDS:

🔧 APPLYING DISCRETIZATION...
✅ DISCRETIZATION COMPLETED!

📊 DISCRETIZED DISTRIBUTIONS:
Efficiency:
  Low: 1906 samples (25.4%)
  Medium: 3912 samples (52.2%)
  High: 1682 samples (22.4%)
Shooting_FG:
  Low: 3229 samples (43.1%)
  Medium: 1545 samples (20.6%)
  High: 2726 samples (36.3%)
Shooting_3PT:
  Low: 3889 samples (51.9%)
  Medium: 750 samples (10.0%)
  High: 2861 samples (38.1%)
Playmaking:
  Low: 1385 samples (18.5%)
  Medium: 2185 samples (29.1%)
  High: 3930 samples (52.4%)
Turnovers:
  Low: 2707 samples (36.1%)
  Medium: 1935 samples (25.8%)
  High: 2858 samples (38.1%)
Offensive_Rebounding:
  Low: 3118 samples (41.6%)
  Medium: 1565 samples (20.9%)
  High: 2817 samples (37.6%)

🔍 SAMPLE SUFFICIENCY CHECK:
✅ Efficiency-

## Phase 2.4: Save Processed Data & Phase Completion

In [10]:
# === PHASE 2.4: SAVE PROCESSED DATA ===
print("=== PHASE 2.4: SAVE PROCESSED DATA ===")

# Verify the final dataset
print("🔍 FINAL DATASET VERIFICATION:")
print(f"📊 Shape: {final_discretized_data.shape}")
print(f"🎯 Columns: {list(final_discretized_data.columns)}")
print(f"📈 Total samples: {len(final_discretized_data)}")

# Check data types and ensure proper categorical encoding
print("\n🔧 DATA TYPE OPTIMIZATION:")
for col in final_discretized_data.columns:
    unique_vals = final_discretized_data[col].unique()
    print(f"  {col}: {list(unique_vals)} - {final_discretized_data[col].dtype}")

# Convert to categorical with logical order for Bayesian network
print("\n🎯 OPTIMIZING FOR BAYESIAN NETWORK:")
final_processed_data = final_discretized_data.copy()

# Ensure consistent categorical ordering
for col in final_processed_data.columns:
    final_processed_data[col] = pd.Categorical(
        final_processed_data[col],
        categories=['Low', 'Medium', 'High'],
        ordered=True
    )

print("✅ All variables encoded as ordered categoricals")

# Final save
final_processed_data.to_csv('nba_lineup_efficiency_final_data.csv', index=False)
print("💾 Saved as 'nba_lineup_efficiency_final_data.csv'")

# Summary statistics
print("\n📋 FINAL DATASET SUMMARY:")
print(f"✅ Samples: {len(final_processed_data):,}")
print(f"✅ Features: {len(final_processed_data.columns)}")
print(f"✅ Data types: All categorical (Low/Medium/High)")
print(f"✅ Basketball logic: Preserved through discretization")
print(f"✅ Ready for Bayesian network training")

print("\n🎉 PHASE 2 COMPLETED SUCCESSFULLY!")
print("🚀 READY FOR PHASE 3: BAYESIAN NETWORK STRUCTURE & LEARNING")

=== PHASE 2.4: SAVE PROCESSED DATA ===
🔍 FINAL DATASET VERIFICATION:
📊 Shape: (7500, 6)
🎯 Columns: ['Efficiency', 'Shooting_FG', 'Shooting_3PT', 'Playmaking', 'Turnovers', 'Offensive_Rebounding']
📈 Total samples: 7500

🔧 DATA TYPE OPTIMIZATION:
  Efficiency: ['Low', 'High', 'Medium'] - category
  Shooting_FG: ['Low', 'Medium', 'High'] - category
  Shooting_3PT: ['Low', 'Medium', 'High'] - category
  Playmaking: ['High', 'Medium', 'Low'] - category
  Turnovers: ['High', 'Medium', 'Low'] - category
  Offensive_Rebounding: ['High', 'Medium', 'Low'] - category

🎯 OPTIMIZING FOR BAYESIAN NETWORK:
✅ All variables encoded as ordered categoricals
💾 Saved as 'nba_lineup_efficiency_final_data.csv'

📋 FINAL DATASET SUMMARY:
✅ Samples: 7,500
✅ Features: 6
✅ Data types: All categorical (Low/Medium/High)
✅ Basketball logic: Preserved through discretization
✅ Ready for Bayesian network training

🎉 PHASE 2 COMPLETED SUCCESSFULLY!
🚀 READY FOR PHASE 3: BAYESIAN NETWORK STRUCTURE & LEARNING


# Phase 3: Bayesian Network Structure & Learning

## Phase 3.1: Design the DAG Structure

In [11]:
# === PHASE 3.1: HIERARCHICAL NETWORK STRUCTURE (RESTART) ===
print("=== PHASE 3.1: HIERARCHICAL NETWORK STRUCTURE ===")

# Load the clean, processed data
print("📥 Loading final processed data...")
final_data = pd.read_csv('nba_lineup_efficiency_final_data.csv')

print(f"📊 Dataset shape: {final_data.shape}")
print(f"🎯 Columns: {list(final_data.columns)}")

# Verify data types are correct for Bayesian network
print("\n🔍 DATA TYPE VERIFICATION:")
for col in final_data.columns:
    print(f"  {col}: {final_data[col].dtype} - {list(final_data[col].unique())}")

# Define the same hierarchical structure
print("\n🔗 DESIGNING HIERARCHICAL STRUCTURE...")
print("🏀 BASKETBALL LOGIC:")
print("  Level 0: Shooting_FG, Shooting_3PT, Playmaking, Turnovers, Offensive_Rebounding")
print("  Level 1: Shooting_Quality ← [FG + 3PT], Ball_Control ← [Playmaking - Turnovers]")
print("  Level 2: Efficiency ← [Shooting_Quality + Ball_Control + Second_Chances]")

from pgmpy.models import DiscreteBayesianNetwork

# Create the hierarchical Bayesian Network structure
hierarchical_model = DiscreteBayesianNetwork([
    # Level 1: Intermediate basketball concepts
    ('Shooting_FG', 'Shooting_Quality'),
    ('Shooting_3PT', 'Shooting_Quality'),
    ('Playmaking', 'Ball_Control'),
    ('Turnovers', 'Ball_Control'),
    ('Offensive_Rebounding', 'Second_Chances'),

    # Level 2: Final efficiency
    ('Shooting_Quality', 'Efficiency'),
    ('Ball_Control', 'Efficiency'),
    ('Second_Chances', 'Efficiency')
])

print("✅ HIERARCHICAL NETWORK STRUCTURE CREATED!")
print(f"📈 Nodes: {hierarchical_model.nodes()}")
print(f"📈 Edges: {hierarchical_model.edges()}")

print("\n🎯 MATHEMATICAL ADVANTAGE:")
print("  • 5 raw skills → 3 intermediate concepts → 1 target")
print("  • Reduces parameter complexity from 729 to 81")
print("  • 8.9x more data-efficient learning!")

print("\n✅ PHASE 3.1 COMPLETED SUCCESSFULLY!")
print("🚀 Ready for Phase 3.2: Learn CPTs with clean data")

=== PHASE 3.1: HIERARCHICAL NETWORK STRUCTURE ===
📥 Loading final processed data...
📊 Dataset shape: (7500, 6)
🎯 Columns: ['Efficiency', 'Shooting_FG', 'Shooting_3PT', 'Playmaking', 'Turnovers', 'Offensive_Rebounding']

🔍 DATA TYPE VERIFICATION:
  Efficiency: object - ['Low', 'High', 'Medium']
  Shooting_FG: object - ['Low', 'Medium', 'High']
  Shooting_3PT: object - ['Low', 'Medium', 'High']
  Playmaking: object - ['High', 'Medium', 'Low']
  Turnovers: object - ['High', 'Medium', 'Low']
  Offensive_Rebounding: object - ['High', 'Medium', 'Low']

🔗 DESIGNING HIERARCHICAL STRUCTURE...
🏀 BASKETBALL LOGIC:
  Level 0: Shooting_FG, Shooting_3PT, Playmaking, Turnovers, Offensive_Rebounding
  Level 1: Shooting_Quality ← [FG + 3PT], Ball_Control ← [Playmaking - Turnovers]
  Level 2: Efficiency ← [Shooting_Quality + Ball_Control + Second_Chances]
✅ HIERARCHICAL NETWORK STRUCTURE CREATED!
📈 Nodes: ['Shooting_FG', 'Shooting_Quality', 'Shooting_3PT', 'Playmaking', 'Ball_Control', 'Turnovers', 'Off

## Phase 3.2: Learn Conditional probability Tables (CPTs)

In [13]:
# === PHASE 3.2 FIXED: BAYESIAN ESTIMATION WITH SMOOTHING ===
print("=== PHASE 3.2 FIXED: BAYESIAN ESTIMATION WITH SMOOTHING ===")

print("🎯 Using Bayesian Estimation for better probability calibration...")

from pgmpy.estimators import BayesianEstimator
from sklearn.metrics import accuracy_score # Import accuracy_score

# Create optimized intermediate variables
print("📊 Creating optimized intermediate variables...")
hierarchical_data = final_data.copy()

def create_optimized_intermediates(data):
    """Create intermediates with better basketball logic"""
    results = data.copy()

    # Use weighted scoring for more precision
    score_map = {'Low': 0, 'Medium': 1, 'High': 2}

    # Shooting: Weight FG% more than 3PT%
    def shooting_quality(row):
        fg_score = score_map[row['Shooting_FG']] * 1.5  # Weight FG% more
        threept_score = score_map[row['Shooting_3PT']] * 1.0
        total = fg_score + threept_score

        if total >= 4.5:  # High threshold
            return 'High'
        elif total <= 1.5:  # Low threshold
            return 'Low'
        else:
            return 'Medium'

    # Ball Control: Strong emphasis on turnover avoidance
    def ball_control(row):
        pm_score = score_map[row['Playmaking']] * 1.0
        to_score = (2 - score_map[row['Turnovers']]) * 1.5  # Weight turnovers heavier
        total = pm_score + to_score

        if total >= 3.5:
            return 'High'
        elif total <= 1.5:
            return 'Low'
        else:
            return 'Medium'

    # Second Chances: Direct but with efficiency guidance
    def second_chances(row):
        return row['Offensive_Rebounding']  # Keep it simple

    results['Shooting_Quality'] = results.apply(shooting_quality, axis=1)
    results['Ball_Control'] = results.apply(ball_control, axis=1)
    results['Second_Chances'] = results.apply(second_chances, axis=1)

    return results

hierarchical_data = create_optimized_intermediates(hierarchical_data)

print("✅ Optimized intermediates created!")
print(f"📊 Enhanced data shape: {hierarchical_data.shape}")

# Learn CPTs with BAYESIAN ESTIMATION (not MLE)
print("\n🎯 LEARNING CPTs WITH BAYESIAN ESTIMATION...")
print("   Using BDeu prior for smoother probability estimates...")

hierarchical_model.fit(
    hierarchical_data,
    estimator=BayesianEstimator,
    prior_type='BDeu',
    equivalent_sample_size=10  # Smoothing parameter
)

print("✅ CPTs learned with Bayesian smoothing!")

# Create inference engine
from pgmpy.inference import VariableElimination
inference = VariableElimination(hierarchical_model)

# Test accuracy with Bayesian estimation
print("📊 TESTING BAYESIAN ESTIMATION ACCURACY...")
bayesian_predictions = []
bayesian_true = []

for idx, row in hierarchical_data.iterrows():
    evidence = {
        'Shooting_FG': row['Shooting_FG'],
        'Shooting_3PT': row['Shooting_3PT'],
        'Playmaking': row['Playmaking'],
        'Turnovers': row['Turnovers'],
        'Offensive_Rebounding': row['Offensive_Rebounding']
    }
    try:
        result = inference.query(variables=['Efficiency'], evidence=evidence)
        predicted = result.state_names['Efficiency'][result.values.argmax()]
        bayesian_predictions.append(predicted)
        bayesian_true.append(row['Efficiency'])
    except:
        continue

bayesian_accuracy = accuracy_score(bayesian_true, bayesian_predictions)

# Check prediction distribution
bayesian_pred_dist = pd.Series(bayesian_predictions).value_counts(normalize=True)

print(f"🎯 BAYESIAN ESTIMATION ACCURACY: {bayesian_accuracy:.1%}")
print(f"📊 PREDICTION DISTRIBUTION: {dict(bayesian_pred_dist)}")

# Compare with previous approaches
print(f"\n📈 ACCURACY COMPARISON:")
print(f"  MLE Hierarchical: 53.9%")
print(f"  MLE Direct: 58.2%")
print(f"  BAYESIAN Hierarchical: {bayesian_accuracy:.1%}")

if bayesian_accuracy > 0.582:
    improvement = (bayesian_accuracy - 0.582) * 100
    print(f"  ✅ IMPROVEMENT: +{improvement:.1f}%")

# Basketball logic validation
print("\n🏀 BAYESIAN MODEL BASKETBALL LOGIC:")
test_cases = [
    ("Elite Shooting", {'Shooting_FG': 'High', 'Shooting_3PT': 'High'}),
    ("Great Ball Control", {'Playmaking': 'High', 'Turnovers': 'Low'}),
    ("Championship Team", {'Shooting_FG': 'High', 'Shooting_3PT': 'High', 'Playmaking': 'High', 'Turnovers': 'Low', 'Offensive_Rebounding': 'High'})
]

for name, evidence in test_cases:
    result = inference.query(variables=['Efficiency'], evidence=evidence)
    high_prob = result.values[result.state_names['Efficiency'].index('High')]
    low_prob = result.values[result.state_names['Efficiency'].index('Low')]
    print(f"  {name}: P(High)={high_prob:.3f}, P(Low)={low_prob:.3f}")

if bayesian_accuracy > 0.65:
    print(f"\n🎉 SUCCESS! Bayesian estimation achieves {bayesian_accuracy:.1%} accuracy!")
    print("🚀 Ready for Phase 3.3 Validation")
else:
    print(f"\n🔧 Bayesian: {bayesian_accuracy:.1%} - Better but needs more work")

print("\n✅ PHASE 3.2 FIXED COMPLETED!")

=== PHASE 3.2 FIXED: BAYESIAN ESTIMATION WITH SMOOTHING ===
🎯 Using Bayesian Estimation for better probability calibration...
📊 Creating optimized intermediate variables...
✅ Optimized intermediates created!
📊 Enhanced data shape: (7500, 9)

🎯 LEARNING CPTs WITH BAYESIAN ESTIMATION...
   Using BDeu prior for smoother probability estimates...
✅ CPTs learned with Bayesian smoothing!
📊 TESTING BAYESIAN ESTIMATION ACCURACY...
🎯 BAYESIAN ESTIMATION ACCURACY: 55.1%
📊 PREDICTION DISTRIBUTION: {'Medium': np.float64(0.7221333333333333), 'High': np.float64(0.1552), 'Low': np.float64(0.12266666666666666)}

📈 ACCURACY COMPARISON:
  MLE Hierarchical: 53.9%
  MLE Direct: 58.2%
  BAYESIAN Hierarchical: 55.1%

🏀 BAYESIAN MODEL BASKETBALL LOGIC:
  Elite Shooting: P(High)=0.429, P(Low)=0.064
  Great Ball Control: P(High)=0.288, P(Low)=0.170
  Championship Team: P(High)=0.555, P(Low)=0.018

🔧 Bayesian: 55.1% - Better but needs more work

✅ PHASE 3.2 FIXED COMPLETED!


## Phase 3.3: initial Model Validation

In [14]:
# === PHASE 3.3: INITIAL MODEL VALIDATION ===
print("=== PHASE 3.3: INITIAL MODEL VALIDATION ===")

# TEST 1: MARGINAL PROBABILITIES
print("\n📊 MARGINAL PROBABILITIES:")
efficiency_marginal = inference.query(variables=['Efficiency'])
print("Overall Efficiency Distribution:")
for state, prob in zip(efficiency_marginal.state_names['Efficiency'], efficiency_marginal.values):
    print(f"  P({state}): {prob:.3f}")

# TEST 2: REAL-WORLD BASKETBALL SCENARIOS
print("\n🏀 REAL-WORLD SCENARIOS:")

# Championship team (elite everything)
print("⭐ CHAMPIONSHIP TEAM (Elite across the board):")
evidence_champ = {
    'Shooting_FG': 'High', 'Shooting_3PT': 'High',
    'Playmaking': 'High', 'Turnovers': 'Low',
    'Offensive_Rebounding': 'High'
}
result_champ = inference.query(variables=['Efficiency'], evidence=evidence_champ)
champ_high = result_champ.values[result_champ.state_names['Efficiency'].index('High')]
print(f"  P(High Efficiency): {champ_high:.3f}")

# Rebuilding team (poor everything)
print("\n🔨 REBUILDING TEAM (Poor across the board):")
evidence_rebuild = {
    'Shooting_FG': 'Low', 'Shooting_3PT': 'Low',
    'Playmaking': 'Low', 'Turnovers': 'High',
    'Offensive_Rebounding': 'Low'
}
result_rebuild = inference.query(variables=['Efficiency'], evidence=evidence_rebuild)
rebuild_low = result_rebuild.values[result_rebuild.state_names['Efficiency'].index('Low')]
print(f"  P(Low Efficiency): {rebuild_low:.3f}")

# TEST 3: ACCURACY CHECK
print("\n🎯 TRAINING ACCURACY CHECK:")
from sklearn.metrics import accuracy_score, classification_report

predictions = []
true_labels = []

for idx, row in hierarchical_data.iterrows():
    evidence = {
        'Shooting_FG': row['Shooting_FG'],
        'Shooting_3PT': row['Shooting_3PT'],
        'Playmaking': row['Playmaking'],
        'Turnovers': row['Turnovers'],
        'Offensive_Rebounding': row['Offensive_Rebounding']
    }
    try:
        result = inference.query(variables=['Efficiency'], evidence=evidence)
        predicted = result.state_names['Efficiency'][result.values.argmax()]
        predictions.append(predicted)
        true_labels.append(row['Efficiency'])
    except:
        continue

accuracy = accuracy_score(true_labels, predictions)
print(f"🎯 TRAINING ACCURACY: {accuracy:.1%}")

print("\n📊 DETAILED PERFORMANCE:")
print(classification_report(true_labels, predictions, target_names=['High', 'Medium', 'Low']))

# Compare with previous attempts
print(f"\n📈 ACCURACY IMPROVEMENT:")
print(f"  Previous Best: 54.9%")
print(f"  Current: {accuracy:.1%}")
if accuracy > 0.549:
    improvement = (accuracy - 0.549) * 100
    print(f"  ✅ IMPROVEMENT: +{improvement:.1f}%")
else:
    print(f"  ⚠️  Still below previous best")

print("\n✅ PHASE 3.3 COMPLETED!")
if accuracy > 0.60:
    print("🚀 EXCELLENT MODEL - Ready for Phase 4!")
else:
    print("🔧 Model needs tuning before Phase 4")

=== PHASE 3.3: INITIAL MODEL VALIDATION ===

📊 MARGINAL PROBABILITIES:
Overall Efficiency Distribution:
  P(High): 0.209
  P(Low): 0.244
  P(Medium): 0.547

🏀 REAL-WORLD SCENARIOS:
⭐ CHAMPIONSHIP TEAM (Elite across the board):
  P(High Efficiency): 0.555

🔨 REBUILDING TEAM (Poor across the board):
  P(Low Efficiency): 0.592

🎯 TRAINING ACCURACY CHECK:
🎯 TRAINING ACCURACY: 55.1%

📊 DETAILED PERFORMANCE:
              precision    recall  f1-score   support

        High       0.52      0.36      0.43      1682
      Medium       0.55      0.26      0.36      1906
         Low       0.56      0.77      0.65      3912

    accuracy                           0.55      7500
   macro avg       0.54      0.47      0.48      7500
weighted avg       0.55      0.55      0.52      7500


📈 ACCURACY IMPROVEMENT:
  Previous Best: 54.9%
  Current: 55.1%
  ✅ IMPROVEMENT: +0.2%

✅ PHASE 3.3 COMPLETED!
🔧 Model needs tuning before Phase 4


## Phase 3.4

In [22]:
# === PHASE 3.4 FIXED: ENSEMBLE BAYESIAN NETWORKS ===
print("=== PHASE 3.4 FIXED: ENSEMBLE BAYESIAN NETWORKS ===\n")

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pgmpy.models import BayesianNetwork, DiscreteBayesianNetwork # Import DiscreteBayesianNetwork
from pgmpy.estimators import BayesianEstimator
from pgmpy.inference import VariableElimination # Import VariableElimination

# Load and prepare data
print("📥 Preparing data for ensemble training...")
data = pd.read_csv('nba_lineup_efficiency_final_data.csv')

# Convert to proper categorical with consistent ordering
for col in data.columns:
    data[col] = pd.Categorical(data[col], categories=['Low', 'Medium', 'High'], ordered=True)

print("🔧 CREATING INTERMEDIATE VARIABLES FOR HIERARCHICAL MODELS...")

# Create intermediate variables for hierarchical models
def create_intermediate_variables(df):
    """Create Shooting_Quality, Ball_Control, and Second_Chances variables"""
    df_extended = df.copy()

    # Define scoring for each level
    score_map = {'Low': 0, 'Medium': 1, 'High': 2}

    # Shooting Quality: Combine FG% and 3PT%
    def get_shooting_quality(row):
        fg_score = score_map[row['Shooting_FG']]
        threept_score = score_map[row['Shooting_3PT']]
        total_score = fg_score + threept_score

        if total_score >= 3:  # High + High = 4, High + Medium = 3
            return 'High'
        elif total_score <= 1:  # Low + Low = 0, Low + Medium = 1
            return 'Low'
        else:
            return 'Medium'

    # Ball Control: Playmaking minus Turnovers (lower turnovers = better)
    def get_ball_control(row):
        playmaking_score = score_map[row['Playmaking']]
        # Invert turnovers: High turnovers = bad, Low turnovers = good
        turnover_score = 2 - score_map[row['Turnovers']]  # Invert the score
        total_score = playmaking_score + turnover_score

        if total_score >= 3:
            return 'High'
        elif total_score <= 1:
            return 'Low'
        else:
            return 'Medium'

    # Second Chances: Direct mapping from offensive rebounding
    def get_second_chances(row):
        return row['Offensive_Rebounding']

    # Apply the functions
    df_extended['Shooting_Quality'] = df_extended.apply(get_shooting_quality, axis=1)
    df_extended['Ball_Control'] = df_extended.apply(get_ball_control, axis=1)
    df_extended['Second_Chances'] = df_extended.apply(get_second_chances, axis=1)

    # Convert to categorical
    df_extended['Shooting_Quality'] = pd.Categorical(df_extended['Shooting_Quality'],
                                                   categories=['Low', 'Medium', 'High'], ordered=True)
    df_extended['Ball_Control'] = pd.Categorical(df_extended['Ball_Control'],
                                               categories=['Low', 'Medium', 'High'], ordered=True)
    df_extended['Second_Chances'] = pd.Categorical(df_extended['Second_Chances'],
                                                 categories=['Low', 'Medium', 'High'], ordered=True)

    return df_extended

# Create extended dataset with intermediate variables *before* splitting
extended_data = create_intermediate_variables(data)
print("✅ Intermediate variables created!")
print(f"📊 Extended dataset columns: {list(extended_data.columns)}")

# Split data for proper validation (use extended_data)
train_data, test_data = train_test_split(extended_data, test_size=0.2, random_state=42, stratify=extended_data['Efficiency'])
print(f"📊 Training samples: {len(train_data)}, Test samples: {len(test_data)}")

# Define multiple network structures
print("\n🔧 CREATING ENSEMBLE OF BAYESIAN NETWORKS...")

ensemble_models = {}

# Model 1: Direct relationships (simplest)
model1 = DiscreteBayesianNetwork([ # Use DiscreteBayesianNetwork
    ('Shooting_FG', 'Efficiency'),
    ('Shooting_3PT', 'Efficiency'),
    ('Playmaking', 'Efficiency'),
    ('Turnovers', 'Efficiency'),
    ('Offensive_Rebounding', 'Efficiency')
])
ensemble_models['Direct_Model'] = model1

# Model 2: Hierarchical with shooting focus
model2 = DiscreteBayesianNetwork([ # Use DiscreteBayesianNetwork
    ('Shooting_FG', 'Shooting_Quality'),
    ('Shooting_3PT', 'Shooting_Quality'),
    ('Shooting_Quality', 'Efficiency'),
    ('Playmaking', 'Efficiency'),
    ('Turnovers', 'Efficiency'),
    ('Offensive_Rebounding', 'Efficiency')
])
ensemble_models['Shooting_Hierarchical'] = model2

# Model 3: Playmaking focused hierarchy
model3 = DiscreteBayesianNetwork([ # Use DiscreteBayesianNetwork
    ('Playmaking', 'Ball_Control'),
    ('Turnovers', 'Ball_Control'),
    ('Ball_Control', 'Efficiency'),
    ('Shooting_FG', 'Efficiency'),
    ('Shooting_3PT', 'Efficiency'),
    ('Offensive_Rebounding', 'Efficiency')
])
ensemble_models['Playmaking_Hierarchical'] = model3

# Model 4: Full hierarchy (your original approach)
model4 = DiscreteBayesianNetwork([ # Use DiscreteBayesianNetwork
    ('Shooting_FG', 'Shooting_Quality'),
    ('Shooting_3PT', 'Shooting_Quality'),
    ('Playmaking', 'Ball_Control'),
    ('Turnovers', 'Ball_Control'),
    ('Offensive_Rebounding', 'Second_Chances'),
    ('Shooting_Quality', 'Efficiency'),
    ('Ball_Control', 'Efficiency'),
    ('Second_Chances', 'Efficiency')
])
ensemble_models['Full_Hierarchy'] = model4

# Model 5: Correlation-based structure (using actual data correlations)
model5 = DiscreteBayesianNetwork([ # Use DiscreteBayesianNetwork
    ('Shooting_FG', 'Efficiency'),      # Strongest correlation
    ('Playmaking', 'Efficiency'),       # Second strongest
    ('Shooting_3PT', 'Efficiency'),     # Third strongest
    ('Turnovers', 'Efficiency'),        # Negative correlation
    ('Offensive_Rebounding', 'Turnovers')  # Weak but meaningful
])
ensemble_models['Correlation_Based'] = model5

# Model 6: Minimal high-impact features only
model6 = DiscreteBayesianNetwork([ # Use DiscreteBayesianNetwork
    ('Shooting_FG', 'Efficiency'),      # Strongest positive correlation
    ('Turnovers', 'Efficiency'),        # Strongest negative correlation
    ('Playmaking', 'Efficiency'),       # Second strongest positive
])
ensemble_models['Minimal_Model'] = model6


print(f"✅ Created {len(ensemble_models)} different Bayesian network structures")

# Train all models and evaluate their performance
print("\n🎯 TRAINING AND EVALUATING ENSEMBLE MODELS...")

model_performances = {}
trained_models = {}

for model_name, model in ensemble_models.items():
    print(f"\n🔧 Training {model_name}...")

    try:
        # Check if all nodes exist in data
        required_nodes = set(model.nodes())
        available_nodes = set(train_data.columns)

        if not required_nodes.issubset(available_nodes):
            missing_nodes = required_nodes - available_nodes
            print(f"   ⚠️  Missing nodes: {missing_nodes}. Skipping...")
            model_performances[model_name] = 0
            continue


        # Fit the model with appropriate data subset
        model.fit(train_data, estimator=BayesianEstimator, equivalent_sample_size=3)
        trained_models[model_name] = model

        # Create inference engine
        inference = VariableElimination(model)

        # Evaluate on test set
        predictions = []
        true_labels = []

        for idx, row in test_data.iterrows():
            # Create evidence with only the nodes this model needs
            evidence = {}
            for node in model.nodes():
                if node != 'Efficiency' and node in row:
                    evidence[node] = row[node]

            try:
                result = inference.query(variables=['Efficiency'], evidence=evidence)
                predicted = result.state_names['Efficiency'][np.argmax(result.values)]
                predictions.append(predicted)
                true_labels.append(row['Efficiency'])
            except Exception as e:
                # If inference fails, use the most common class as fallback
                predictions.append('Medium')
                true_labels.append(row['Efficiency'])
                continue

        accuracy = accuracy_score(true_labels, predictions)
        model_performances[model_name] = accuracy
        print(f"   ✅ {model_name} Accuracy: {accuracy:.1%}")

    except Exception as e:
        print(f"   ❌ {model_name} failed: {e}")
        model_performances[model_name] = 0


print("\n📊 ENSEMBLE PERFORMANCE SUMMARY:")
for model_name, accuracy in sorted(model_performances.items(), key=lambda x: x[1], reverse=True):
    print(f"   {model_name}: {accuracy:.1%}")

# Select the best model
best_model_name = max(model_performances, key=model_performances.get)
best_model = trained_models.get(best_model_name) # Use .get to handle cases where a model failed training
best_accuracy = model_performances[best_model_name]

print(f"\n🏆 BEST MODEL: {best_model_name} ({best_accuracy:.1%} accuracy)")

# Create ensemble predictions (majority voting) only from successful models
print("\n🤝 CREATING ENSEMBLE PREDICTIONS (Majority Voting)...")

def ensemble_predict(evidence_data, models_dict, performances_dict):
    """Get ensemble prediction using weighted voting"""
    all_predictions = []
    valid_models = []

    for model_name, model in models_dict.items():
        if model_name in performances_dict and performances_dict[model_name] > 0.5: # Only use decent models
            try:
                inference = VariableElimination(model)
                predictions = []

                for idx, row in evidence_data.iterrows():
                    evidence = {}
                    for node in model.nodes():
                        if node != 'Efficiency' and node in row: # Ensure node is in the evidence data
                             evidence[node] = row[node]

                    result = inference.query(variables=['Efficiency'], evidence=evidence)
                    predicted = result.state_names['Efficiency'][np.argmax(result.values)]
                    predictions.append(predicted)

                all_predictions.append(predictions)
                valid_models.append(model_name)
            except:
                continue

    if not all_predictions:
        print("   ⚠️  No valid models for ensemble prediction")
        return None

    print(f"   ✅ Using {len(valid_models)} models: {valid_models}")

    # Weighted majority voting based on individual model performance
    ensemble_final = []
    for i in range(len(evidence_data)):
        votes = {'Low': 0, 'Medium': 0, 'High': 0}
        for j, pred_list in enumerate(all_predictions):
            model_name = valid_models[j]
            weight = performances_dict.get(model_name, 0.5) # Use accuracy as weight
            votes[pred_list[i]] += weight

        # Select the class with highest weighted votes
        winner = max(votes.items(), key=lambda x: x[1])[0]
        ensemble_final.append(winner)

    return ensemble_final

# Test ensemble performance
ensemble_predictions = ensemble_predict(test_data, trained_models, model_performances)
if ensemble_predictions:
    ensemble_accuracy = accuracy_score(test_data['Efficiency'], ensemble_predictions)
    print(f"🎯 ENSEMBLE ACCURACY: {ensemble_accuracy:.1%}")

    print(f"\n📈 PERFORMANCE COMPARISON:")
    print(f"  Single Best Model: {best_accuracy:.1%}")
    print(f"  Ensemble Method: {ensemble_accuracy:.1%}")
    improvement = (ensemble_accuracy - best_accuracy) * 100
    print(f"  📈 ENSEMBLE IMPROVEMENT: +{improvement:.1f}%")
else:
    print("❌ Ensemble prediction failed")


# Save the best performing models
import pickle
if trained_models: # Only save if at least one model trained successfully
    ensemble_data = {
        'models': trained_models,
        'performances': model_performances,
        'best_model': best_model_name,
        'ensemble_accuracy': ensemble_accuracy if ensemble_predictions else best_accuracy # Save ensemble accuracy if available
    }

    with open('bayesian_ensemble_fixed.pkl', 'wb') as f:
        pickle.dump(ensemble_data, f)

    print("💾 Saved ensemble models as 'bayesian_ensemble_fixed.pkl'")

print("\n✅ PHASE 3.4 FIXED COMPLETED!")

=== PHASE 3.4 FIXED: ENSEMBLE BAYESIAN NETWORKS ===

📥 Preparing data for ensemble training...
🔧 CREATING INTERMEDIATE VARIABLES FOR HIERARCHICAL MODELS...
✅ Intermediate variables created!
📊 Extended dataset columns: ['Efficiency', 'Shooting_FG', 'Shooting_3PT', 'Playmaking', 'Turnovers', 'Offensive_Rebounding', 'Shooting_Quality', 'Ball_Control', 'Second_Chances']
📊 Training samples: 6000, Test samples: 1500

🔧 CREATING ENSEMBLE OF BAYESIAN NETWORKS...
✅ Created 6 different Bayesian network structures

🎯 TRAINING AND EVALUATING ENSEMBLE MODELS...

🔧 Training Direct_Model...
   ✅ Direct_Model Accuracy: 55.4%

🔧 Training Shooting_Hierarchical...
   ✅ Shooting_Hierarchical Accuracy: 53.4%

🔧 Training Playmaking_Hierarchical...
   ✅ Playmaking_Hierarchical Accuracy: 56.6%

🔧 Training Full_Hierarchy...
   ✅ Full_Hierarchy Accuracy: 55.7%

🔧 Training Correlation_Based...
   ✅ Correlation_Based Accuracy: 56.1%

🔧 Training Minimal_Model...
   ✅ Minimal_Model Accuracy: 56.8%

📊 ENSEMBLE PERFO

## Phase 3.5

In [20]:
# === PHASE 3.5: ADVANCED ENSEMBLE WITH PROBABILITY FUSION ===
print("=== PHASE 3.5: ADVANCED ENSEMBLE WITH PROBABILITY FUSION ===\n")

print("🔧 IMPLEMENTING PROBABILITY-BASED ENSEMBLE FUSION...")

def probability_fusion_ensemble(evidence_data, models_dict, performances_dict):
    """Advanced ensemble using probability fusion instead of hard voting"""
    all_probabilities = []
    model_weights = []

    for model_name, model in models_dict.items():
        if model_name in performances_dict and performances_dict[model_name] > 0:
            try:
                inference = VariableElimination(model)
                model_probs = []

                for idx, row in evidence_data.iterrows():
                    evidence = {col: row[col] for col in model.nodes() if col != 'Efficiency'}
                    result = inference.query(variables=['Efficiency'], evidence=evidence)

                    # Get probability distribution
                    prob_dict = {
                        'Low': result.values[result.state_names['Efficiency'].index('Low')],
                        'Medium': result.values[result.state_names['Efficiency'].index('Medium')],
                        'High': result.values[result.state_names['Efficiency'].index('High')]
                    }
                    model_probs.append(prob_dict)

                all_probabilities.append(model_probs)
                model_weights.append(performances_dict[model_name])  # Use accuracy as weight

            except Exception as e:
                print(f"   ⚠️ {model_name} probability fusion failed: {e}")
                continue

    if not all_probabilities:
        return None

    # Weighted probability fusion
    final_predictions = []
    for i in range(len(evidence_data)):
        fused_probs = {'Low': 0, 'Medium': 0, 'High': 0}
        total_weight = sum(model_weights)

        for j, probs_list in enumerate(all_probabilities):
            weight = model_weights[j] / total_weight  # Normalize weights
            for state in ['Low', 'Medium', 'High']:
                fused_probs[state] += probs_list[i][state] * weight

        # Select class with highest fused probability
        winner = max(fused_probs.items(), key=lambda x: x[1])[0]
        final_predictions.append(winner)

    return final_predictions

# Test probability fusion ensemble
print("🎯 TESTING PROBABILITY FUSION ENSEMBLE...")
fusion_predictions = probability_fusion_ensemble(test_data, trained_models, model_performances)

if fusion_predictions:
    fusion_accuracy = accuracy_score(test_data['Efficiency'], fusion_predictions)
    print(f"🎯 PROBABILITY FUSION ACCURACY: {fusion_accuracy:.1%}")

print(f"\\n📊 FINAL ENSEMBLE COMPARISON:")
print(f"  Single Best Model: {best_accuracy:.1%}")
print(f"  Majority Voting Ensemble: {ensemble_accuracy:.1%}")
print(f"  Probability Fusion Ensemble: {fusion_accuracy:.1%}")

# Test on challenging cases
print("\n🏀 TESTING ON CHALLENGING BASKETBALL SCENARIOS...")

test_cases = [
    {
        'name': 'Elite Shooting, Poor Defense',
        'evidence': {'Shooting_FG': 'High', 'Shooting_3PT': 'High', 'Playmaking': 'Low', 'Turnovers': 'High', 'Offensive_Rebounding': 'Low'}
    },
    {
        'name': 'Great Defense, Poor Offense',
        'evidence': {'Shooting_FG': 'Low', 'Shooting_3PT': 'Low', 'Playmaking': 'High', 'Turnovers': 'Low', 'Offensive_Rebounding': 'High'}
    },
    {
        'name': 'Balanced Team',
        'evidence': {'Shooting_FG': 'Medium', 'Shooting_3PT': 'Medium', 'Playmaking': 'Medium', 'Turnovers': 'Medium', 'Offensive_Rebounding': 'Medium'}
    }
]

print("\\n🔍 ENSEMBLE PREDICTIONS FOR TEST CASES:")
for case in test_cases:
    # Convert to DataFrame for ensemble prediction
    case_df = pd.DataFrame([case['evidence']])

    ensemble_pred = ensemble_predict(case_df, trained_models, model_performances)
    fusion_pred = probability_fusion_ensemble(case_df, trained_models, model_performances)

    print(f"\\n  {case['name']}:")
    print(f"    Evidence: {case['evidence']}")
    print(f"    Majority Voting: {ensemble_pred[0] if ensemble_pred else 'N/A'}")
    print(f"    Probability Fusion: {fusion_pred[0] if fusion_pred else 'N/A'}")

# Feature importance analysis
print("\n📈 FEATURE IMPORTANCE ANALYSIS...")

def analyze_feature_importance(model, feature_names):
    """Analyze how features affect efficiency predictions"""
    importance_scores = {}

    for feature in feature_names:
        if feature != 'Efficiency':
            # Test probability changes when feature changes
            base_evidence = {f: 'Medium' for f in feature_names if f != 'Efficiency'}

            # Query with feature set to Low
            base_evidence[feature] = 'Low'
            result_low = model.query(variables=['Efficiency'], evidence=base_evidence)
            prob_low_high = result_low.values[result_low.state_names['Efficiency'].index('High')]

            # Query with feature set to High
            base_evidence[feature] = 'High'
            result_high = model.query(variables=['Efficiency'], evidence=base_evidence)
            prob_high_high = result_high.values[result_high.state_names['Efficiency'].index('High')]

            # Importance = probability difference
            importance = abs(prob_high_high - prob_low_high)
            importance_scores[feature] = importance

    return importance_scores

# Analyze best model's feature importance
if best_model_name in trained_models:
    features = [node for node in best_model.nodes() if node != 'Efficiency']
    importance = analyze_feature_importance(VariableElimination(best_model), features)

    print("🔍 FEATURE IMPORTANCE IN BEST MODEL:")
    for feature, score in sorted(importance.items(), key=lambda x: x[1], reverse=True):
        print(f"   {feature}: {score:.3f}")

print("\\n✅ PHASE 3.5 COMPLETED!")
print("🚀 ENSEMBLE METHOD READY FOR DEPLOYMENT!")

=== PHASE 3.5: ADVANCED ENSEMBLE WITH PROBABILITY FUSION ===

🔧 IMPLEMENTING PROBABILITY-BASED ENSEMBLE FUSION...
🎯 TESTING PROBABILITY FUSION ENSEMBLE...
🎯 PROBABILITY FUSION ACCURACY: 56.9%
\n📊 FINAL ENSEMBLE COMPARISON:
  Single Best Model: 56.1%
  Majority Voting Ensemble: 56.1%
  Probability Fusion Ensemble: 56.9%

🏀 TESTING ON CHALLENGING BASKETBALL SCENARIOS...
\n🔍 ENSEMBLE PREDICTIONS FOR TEST CASES:
\n  Elite Shooting, Poor Defense:
    Evidence: {'Shooting_FG': 'High', 'Shooting_3PT': 'High', 'Playmaking': 'Low', 'Turnovers': 'High', 'Offensive_Rebounding': 'Low'}
    Majority Voting: Medium
    Probability Fusion: Medium
\n  Great Defense, Poor Offense:
    Evidence: {'Shooting_FG': 'Low', 'Shooting_3PT': 'Low', 'Playmaking': 'High', 'Turnovers': 'Low', 'Offensive_Rebounding': 'High'}
    Majority Voting: Medium
    Probability Fusion: Medium
\n  Balanced Team:
    Evidence: {'Shooting_FG': 'Medium', 'Shooting_3PT': 'Medium', 'Playmaking': 'Medium', 'Turnovers': 'Medium', 'O

In [24]:
# ==================================================
# NBA LINEUP EFFICIENCY BAYESIAN NETWORK - OPTIMIZED
# ==================================================

# Install required packages
!pip install pgmpy pandas numpy matplotlib seaborn networkx nba_api scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from pgmpy.inference import VariableElimination
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

print("✅ Packages imported successfully!")

# ==================================================
# PHASE 1: DATA ACQUISITION & INTEGRATION (KEEPING WORKING PART)
# ==================================================

print("🚀 GETTING REAL NBA LINEUP DATA FROM OFFICIAL NBA API...")

from nba_api.stats.endpoints import teamdashlineups
from nba_api.stats.static import teams
import time

# Get all NBA teams
nba_teams = teams.get_teams()
team_dict = {team['full_name']: team['id'] for team in nba_teams}

print(f"✅ Found {len(team_dict)} NBA teams")

# Function to get lineups for a team
def get_lineups(team_id_i):
    try:
        lineup = teamdashlineups.TeamDashLineups(
            team_id=team_id_i,
            season='2023-24',
            season_type_all_star='Regular Season',
            group_quantity=5,
            per_mode_detailed='Totals'
        )
        df = lineup.get_data_frames()
        return df[1]  # Lineup data
    except Exception as e:
        print(f"❌ Error getting lineups for team {team_id_i}: {e}")
        return None

# Get lineups for all teams
print("\n📥 DOWNLOADING LINEUP DATA FOR ALL TEAMS...")
dataframes = []

for i, team_name in enumerate(team_dict.keys()):
    team_id_i = team_dict[team_name]
    print(f"   {i+1}/{len(team_dict)}: Getting {team_name}...")

    team_lineup = get_lineups(team_id_i)
    if team_lineup is not None and not team_lineup.empty:
        team_lineup['team'] = team_name
        team_lineup['team_id'] = team_id_i
        dataframes.append(team_lineup)

    time.sleep(0.5)  # Rate limiting

# Combine all team lineups
if dataframes:
    league_lineup = pd.concat(dataframes, ignore_index=True)
    league_lineup['players_list'] = league_lineup['GROUP_NAME'].str.split(' - ')

    print(f"\n✅ SUCCESS: Downloaded {len(league_lineup)} lineup combinations!")
    league_lineup.to_csv('nba_lineups_2024_api.csv', index=False)

    # Show sample
    print("\n🔍 SAMPLE OF REAL NBA LINEUP DATA:")
    display(league_lineup[['GROUP_NAME', 'team', 'MIN', 'PLUS_MINUS', 'FG_PCT', 'FG3_PCT']].head(3))
else:
    print("❌ No lineup data could be downloaded")

# ==================================================
# PHASE 2: NEW DATA PROCESSING APPROACH
# ==================================================

print("\n=== PHASE 2: OPTIMIZED DATA PROCESSING ===")

# Load the data
lineup_data = pd.read_csv('nba_lineups_2024_api.csv')
print(f"📊 Original data shape: {lineup_data.shape}")

# Create rate-based features (per 48 minutes)
def create_advanced_features(data):
    df = data.copy()
    minutes = df['MIN']

    # Efficiency metrics
    df['Efficiency'] = df['PLUS_MINUS']

    # Shooting metrics (already percentages)
    df['Shooting_FG'] = df['FG_PCT']
    df['Shooting_3PT'] = df['FG3_PCT']

    # Playmaking and ball control (per 48 min)
    df['Playmaking'] = (df['AST'] / minutes) * 48
    df['Turnovers'] = (df['TOV'] / minutes) * 48

    # Rebounding and defense (per 48 min)
    df['Offensive_Rebounding'] = (df['OREB'] / minutes) * 48
    df['Defensive_Rebounding'] = (df['DREB'] / minutes) * 48

    return df[['Efficiency', 'Shooting_FG', 'Shooting_3PT',
               'Playmaking', 'Turnovers', 'Offensive_Rebounding', 'Defensive_Rebounding']]

# Create advanced features
advanced_data = create_advanced_features(lineup_data)
advanced_data = advanced_data.replace([np.inf, -np.inf], np.nan).dropna()

print(f"📊 Advanced features shape: {advanced_data.shape}")

# Check correlations
print("\n📊 FEATURE CORRELATIONS WITH EFFICIENCY:")
correlations = advanced_data.corr()['Efficiency'].sort_values(ascending=False)
for feature, corr in correlations.items():
    if feature != 'Efficiency':
        print(f"   {feature}: {corr:.3f}")

# ==================================================
# NEW DISCRETIZATION STRATEGY
# ==================================================

print("\n🔧 SMART DISCRETIZATION WITH QUANTILES...")

def smart_discretize(data):
    df = data.copy()

    # Use quantile-based discretization for better distribution
    discretized_data = pd.DataFrame()

    # Efficiency: Use basketball-informed thresholds
    discretized_data['Efficiency'] = pd.cut(df['Efficiency'],
                                          bins=[-float('inf'), -3, 3, float('inf')],
                                          labels=['Low', 'Medium', 'High'])

    # Shooting: Use performance-based quantiles
    discretized_data['Shooting_FG'] = pd.qcut(df['Shooting_FG'], q=3, labels=['Low', 'Medium', 'High'])
    discretized_data['Shooting_3PT'] = pd.qcut(df['Shooting_3PT'], q=3, labels=['Low', 'Medium', 'High'])

    # Playmaking and turnovers: Use rate-based quantiles
    discretized_data['Playmaking'] = pd.qcut(df['Playmaking'], q=3, labels=['Low', 'Medium', 'High'])
    discretized_data['Turnovers'] = pd.qcut(df['Turnovers'], q=3, labels=['Low', 'Medium', 'High'])

    # Rebounding: Use rate-based quantiles
    discretized_data['Offensive_Rebounding'] = pd.qcut(df['Offensive_Rebounding'], q=3, labels=['Low', 'Medium', 'High'])
    discretized_data['Defensive_Rebounding'] = pd.qcut(df['Defensive_Rebounding'], q=3, labels=['Low', 'Medium', 'High'])

    return discretized_data

# Apply discretization
final_data = smart_discretize(advanced_data)
print(f"📊 Discretized data shape: {final_data.shape}")

# Check distribution
print("\n📊 DISCRETIZED DISTRIBUTION:")
for col in final_data.columns:
    dist = final_data[col].value_counts(normalize=True)
    print(f"{col}: {dict(dist)}")

# Save processed data
final_data.to_csv('optimized_nba_data.csv', index=False)
print("💾 Saved optimized data")

# ==================================================
# PHASE 3: NEW BAYESIAN NETWORK STRUCTURE
# ==================================================

print("\n=== PHASE 3: OPTIMIZED BAYESIAN NETWORK ===")

# Load processed data
final_data = pd.read_csv('optimized_nba_data.csv')
print(f"📊 Training data: {final_data.shape}")

# ==================================================
# STRATEGY 1: SIMPLIFIED DIRECT STRUCTURE
# ==================================================

print("\n🎯 STRATEGY 1: SIMPLIFIED DIRECT STRUCTURE")

# Create a simpler, more direct network
simple_model = DiscreteBayesianNetwork([
    # Direct influences on efficiency
    ('Shooting_FG', 'Efficiency'),
    ('Shooting_3PT', 'Efficiency'),
    ('Playmaking', 'Efficiency'),
    ('Turnovers', 'Efficiency'),
    ('Offensive_Rebounding', 'Efficiency'),
    ('Defensive_Rebounding', 'Efficiency')
])

print("✅ Simple direct structure created")

# Learn CPTs with Bayesian estimation
simple_model.fit(final_data, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=5)
print("✅ CPTs learned with Bayesian estimation")

# Create inference engine
inference_simple = VariableElimination(simple_model)

# Test accuracy
print("\n📊 TESTING SIMPLE MODEL ACCURACY...")
predictions_simple = []
true_labels = []

for idx, row in final_data.iterrows():
    evidence = {col: row[col] for col in final_data.columns if col != 'Efficiency'}
    try:
        result = inference_simple.query(variables=['Efficiency'], evidence=evidence)
        predicted = result.state_names['Efficiency'][np.argmax(result.values)]
        predictions_simple.append(predicted)
        true_labels.append(row['Efficiency'])
    except:
        continue

accuracy_simple = accuracy_score(true_labels, predictions_simple)
print(f"🎯 SIMPLE MODEL ACCURACY: {accuracy_simple:.1%}")

# ==================================================
# STRATEGY 2: CAUSAL CHAIN STRUCTURE
# ==================================================

print("\n🎯 STRATEGY 2: CAUSAL CHAIN STRUCTURE")

# Create causal chain: Shooting → Ball Handling → Rebounding → Efficiency
causal_model = DiscreteBayesianNetwork([
    # Shooting influences
    ('Shooting_FG', 'Shooting_3PT'),  # Good shooters tend to be good at both

    # Ball handling chain
    ('Playmaking', 'Turnovers'),      # More playmaking can lead to more turnovers
    ('Turnovers', 'Efficiency'),      # Turnovers directly hurt efficiency

    # Rebounding influences
    ('Offensive_Rebounding', 'Defensive_Rebounding'),  # Good rebounders do both

    # Direct efficiency influences
    ('Shooting_FG', 'Efficiency'),
    ('Shooting_3PT', 'Efficiency'),
    ('Playmaking', 'Efficiency'),
    ('Offensive_Rebounding', 'Efficiency'),
    ('Defensive_Rebounding', 'Efficiency')
])

print("✅ Causal chain structure created")

# Learn CPTs
causal_model.fit(final_data, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=5)
print("✅ CPTs learned")

# Test accuracy
inference_causal = VariableElimination(causal_model)

predictions_causal = []
for idx, row in final_data.iterrows():
    evidence = {col: row[col] for col in final_data.columns if col != 'Efficiency'}
    try:
        result = inference_causal.query(variables=['Efficiency'], evidence=evidence)
        predicted = result.state_names['Efficiency'][np.argmax(result.values)]
        predictions_causal.append(predicted)
    except:
        continue

accuracy_causal = accuracy_score(true_labels[:len(predictions_causal)], predictions_causal)
print(f"🎯 CAUSAL MODEL ACCURACY: {accuracy_causal:.1%}")

# ==================================================
# STRATEGY 3: ENSEMBLE APPROACH
# ==================================================

print("\n🎯 STRATEGY 3: ENSEMBLE PREDICTION")

def ensemble_predict(row, models, inferences):
    predictions = []

    for model, inference in zip(models, inferences):
        evidence = {col: row[col] for col in final_data.columns if col != 'Efficiency'}
        try:
            result = inference.query(variables=['Efficiency'], evidence=evidence)
            predicted = result.state_names['Efficiency'][np.argmax(result.values)]
            predictions.append(predicted)
        except:
            continue

    # Return most frequent prediction
    if predictions:
        return max(set(predictions), key=predictions.count)
    else:
        return 'Medium'  # Default

# Create ensemble
models = [simple_model, causal_model]
inferences = [inference_simple, inference_causal]

ensemble_predictions = []
for idx, row in final_data.iterrows():
    pred = ensemble_predict(row, models, inferences)
    ensemble_predictions.append(pred)

accuracy_ensemble = accuracy_score(true_labels, ensemble_predictions)
print(f"🎯 ENSEMBLE ACCURACY: {accuracy_ensemble:.1%}")

# ==================================================
# FINAL RESULTS & ANALYSIS
# ==================================================

print("\n" + "="*50)
print("FINAL RESULTS SUMMARY")
print("="*50)

print(f"📊 Dataset Size: {len(final_data):,} samples")
print(f"🎯 Simple Direct Model: {accuracy_simple:.1%}")
print(f"🎯 Causal Chain Model: {accuracy_causal:.1%}")
print(f"🎯 Ensemble Model: {accuracy_ensemble:.1%}")

# Find best model
accuracies = {
    'Simple Direct': accuracy_simple,
    'Causal Chain': accuracy_causal,
    'Ensemble': accuracy_ensemble
}

best_model_name = max(accuracies, key=accuracies.get)
best_accuracy = accuracies[best_model_name]

print(f"\n🏆 BEST MODEL: {best_model_name} ({best_accuracy:.1%})")

if best_accuracy > 0.54:
    improvement = (best_accuracy - 0.54) * 100
    print(f"✅ IMPROVEMENT: +{improvement:.1f}% over previous approach!")

# Detailed performance analysis
print(f"\n📈 DETAILED PERFORMANCE (Best Model):")
if best_model_name == 'Simple Direct':
    best_predictions = predictions_simple
elif best_model_name == 'Causal Chain':
    best_predictions = predictions_causal
else:
    best_predictions = ensemble_predictions

print(classification_report(true_labels, best_predictions, target_names=['Low', 'Medium', 'High']))

# ==================================================
# BASKETBALL INSIGHTS
# ==================================================

print("\n🏀 BASKETBALL INSIGHTS FROM BEST MODEL:")

# Test championship scenario
print("\n⭐ CHAMPIONSHIP TEAM ANALYSIS:")
champ_evidence = {
    'Shooting_FG': 'High', 'Shooting_3PT': 'High',
    'Playmaking': 'High', 'Turnovers': 'Low',
    'Offensive_Rebounding': 'High', 'Defensive_Rebounding': 'High'
}

if best_model_name == 'Simple Direct':
    result = inference_simple.query(variables=['Efficiency'], evidence=champ_evidence)
elif best_model_name == 'Causal Chain':
    result = inference_causal.query(variables=['Efficiency'], evidence=champ_evidence)
else:
    # Use simple model for insight generation
    result = inference_simple.query(variables=['Efficiency'], evidence=champ_evidence)

print("Elite team probabilities:")
for state, prob in zip(result.state_names['Efficiency'], result.values):
    print(f"  P(Efficiency = {state}): {prob:.3f}")

# Feature importance analysis
print("\n🔍 FEATURE IMPORTANCE ANALYSIS:")
base_case = {col: 'Medium' for col in final_data.columns if col != 'Efficiency'}

for feature in ['Shooting_FG', 'Shooting_3PT', 'Playmaking', 'Turnovers']:
    # Test improvement when feature goes from Low to High
    evidence_low = base_case.copy()
    evidence_low[feature] = 'Low'

    evidence_high = base_case.copy()
    evidence_high[feature] = 'High'

    if best_model_name == 'Simple Direct':
        result_low = inference_simple.query(variables=['Efficiency'], evidence=evidence_low)
        result_high = inference_simple.query(variables=['Efficiency'], evidence=evidence_high)
    else:
        result_low = inference_causal.query(variables=['Efficiency'], evidence=evidence_low)
        result_high = inference_causal.query(variables=['Efficiency'], evidence=evidence_high)

    p_high_low = result_low.values[result_low.state_names['Efficiency'].index('High')]
    p_high_high = result_high.values[result_high.state_names['Efficiency'].index('High')]

    improvement = p_high_high - p_high_low
    print(f"  {feature}: +{improvement:.3f} P(High) when going from Low→High")

print(f"\n🎉 OPTIMIZATION COMPLETE!")
print(f"🚀 Best model achieves {best_accuracy:.1%} accuracy")
print("💡 Ready for deployment and further analysis!")

✅ Packages imported successfully!
🚀 GETTING REAL NBA LINEUP DATA FROM OFFICIAL NBA API...
✅ Found 30 NBA teams

📥 DOWNLOADING LINEUP DATA FOR ALL TEAMS...
   1/30: Getting Atlanta Hawks...
   2/30: Getting Boston Celtics...
   3/30: Getting Cleveland Cavaliers...
   4/30: Getting New Orleans Pelicans...
   5/30: Getting Chicago Bulls...
   6/30: Getting Dallas Mavericks...
   7/30: Getting Denver Nuggets...
   8/30: Getting Golden State Warriors...
   9/30: Getting Houston Rockets...
   10/30: Getting Los Angeles Clippers...
   11/30: Getting Los Angeles Lakers...
   12/30: Getting Miami Heat...
   13/30: Getting Milwaukee Bucks...
   14/30: Getting Minnesota Timberwolves...
   15/30: Getting Brooklyn Nets...
   16/30: Getting New York Knicks...
   17/30: Getting Orlando Magic...
   18/30: Getting Indiana Pacers...
   19/30: Getting Philadelphia 76ers...
   20/30: Getting Phoenix Suns...
   21/30: Getting Portland Trail Blazers...
   22/30: Getting Sacramento Kings...
   23/30: Getting

,GROUP_NAME,team,MIN,PLUS_MINUS,FG_PCT,FG3_PCT
0,C. Capela - D. Murray - T. Young - S. Bey - J....,Atlanta Hawks,288.680000,-88.0,0.446,0.312
1,C. Capela - D. Murray - T. Young - D. Hunter -...,Atlanta Hawks,176.911667,8.0,0.468,0.384
2,C. Capela - D. Murray - T. Young - D. Hunter -...,Atlanta Hawks,171.505000,-26.0,0.464,0.367



=== PHASE 2: OPTIMIZED DATA PROCESSING ===
📊 Original data shape: (7500, 59)
📊 Advanced features shape: (7500, 7)

📊 FEATURE CORRELATIONS WITH EFFICIENCY:
   Shooting_FG: 0.329
   Playmaking: 0.274
   Defensive_Rebounding: 0.243
   Shooting_3PT: 0.223
   Offensive_Rebounding: 0.022
   Turnovers: -0.141

🔧 SMART DISCRETIZATION WITH QUANTILES...
📊 Discretized data shape: (7500, 7)

📊 DISCRETIZED DISTRIBUTION:
Efficiency: {'Low': np.float64(0.35306666666666664), 'Medium': np.float64(0.3368), 'High': np.float64(0.3101333333333333)}
Shooting_FG: {'Low': np.float64(0.3374666666666667), 'High': np.float64(0.3333333333333333), 'Medium': np.float64(0.3292)}
Shooting_3PT: {'Low': np.float64(0.3456), 'High': np.float64(0.3284), 'Medium': np.float64(0.326)}
Playmaking: {'Medium': np.float64(0.3336), 'Low': np.float64(0.3334666666666667), 'High': np.float64(0.33293333333333336)}
Turnovers: {'Low': np.float64(0.3333333333333333), 'Medium': np.float64(0.3333333333333333), 'High': np.float64(0.333333